In [ ]:
import json
import glob
import os
from functools import reduce

import numpy as np
from matplotlib import pyplot as plt

from enum import IntEnum
from dataclasses import dataclass, field
from typing import List
from copy import deepcopy

from scobility import *

In [ ]:
itl = Tournament()

# https://github.com/G-Wen/itl_history

# Song info
song_files = glob.glob('itl_data/song_info/*.json')
for fn in song_files:
    with open(fn, 'r') as fp:
        song_data = json.load(fp)
        s = Song(song_data['song'])
        itl.songs[s.s_id] = s

# Player info
player_files = glob.glob('itl_data/entrant_info/*.json')
for fn in player_files:
    with open(fn, 'r') as fp:
        player_data = json.load(fp)
        p = Player(player_data['entrant'])
        itl.players[p.e_id] = p

# Score data
score_files = glob.glob('itl_data/song_scores/*.json')
for fn in score_files:
    with open(fn, 'r') as fp:
        score_data = json.load(fp)
        itl.load_score_data(score_data['scores'])

In [ ]:
# Calculate relationships
itl.setup_relationships()
itl.calc_relationships(verbal=False)

In [ ]:
itl.order_songs_initial(verbal=True, visual=False)

In [ ]:
# A few iterations of plain bubble sort until the worst un-ordered offenders
# are smoothed out.
for a in range(10):
    prev_ordering = [s for s in itl.ordering]
    for i in range(len(itl.ordering)-1):
        # Bubble sort!
        x = itl.ordering[i]
        y = itl.ordering[i+1]
        r_fwd = itl.relationship_lookup(x, y, allow_link=True)
        r_rev = itl.relationship_lookup(y, x, allow_link=True)
        if r_fwd and r_fwd.relation < Tournament.MONO_THRESHOLD:
            if r_rev and r_rev.relation > r_fwd.relation:
                # Only swap if an improvement would be observed!
                itl.ordering = itl.ordering[:i] + [y, x] + itl.ordering[i+2:]
    for i, (prev, next) in enumerate(zip(prev_ordering, itl.ordering)):
        if prev.s_id != next.s_id:
            print(f'#{i:3d}: {prev} -> {next}')

In [ ]:
# checking my work haha
order_progressive_rel = []
for i_song in range(len(itl.ordering)-1):
    order_progressive_rel.append(itl.relationship_lookup(itl.ordering[i_song], itl.ordering[i_song+1]))
for r in order_progressive_rel:
    print(f"{r.pair_title():50s}:\n>>> corr. {r.relation:0.6f} from {len(r.e_common)} players")

plt.plot([r.relation for r in order_progressive_rel])
plt.plot([1 for r in order_progressive_rel])
plt.show()

In [ ]:
# Generate the scobility rating for each chart.
m = 2               # Minimum scobility is 1.0
for i, s in enumerate(itl.ordering):
    print(f"{np.log2(m):5.1f}🌶️ {str(s):>60s}")
    if i < len(itl.ordering)-1:
        r = itl.relationship_lookup(itl.ordering[i], itl.ordering[i+1])
        m *= r.relation

In [ ]:
# Generate the naive scobility rating for each chart.
scobility = []
for i, s in enumerate(itl.ordering):
    if i == 0:
        scobility = [1]               # Minimum scobility is 0.0
        continue
    r = itl.relationship_lookup(itl.ordering[i-1], itl.ordering[i])
    scobility.append(scobility[-1] * r.relation)

for i, s in enumerate(itl.ordering):
    print(f"{np.log2(scobility[i]):5.1f}🌶️ {str(s):>60s}")

In [ ]:
# Converge the scobility rating by comparing with charts that are "close".

scobility_rev = [v for v in scobility]
ordering_rev = [s for s in itl.ordering]
check_window_lower = 10
check_window_upper = 6
scobility_iter = 500
for k in range(scobility_iter):
    ordering_last = [v for v in ordering_rev]
    for i, s in enumerate(ordering_rev):
        window = ordering_rev[max(0, i - check_window_lower) : min(i + check_window_upper + 1, len(ordering_rev))]
        nearby_scobility = scobility_rev[max(0, i - check_window_lower) : min(i + check_window_upper + 1, len(ordering_rev))]
        rel_window = [itl.relationship_lookup(w, s) for w in window]
        nearby_relation = [r.relation for r in rel_window]
        nearby_strength = [r.strength for r in rel_window]
        nearby_contrib = [r * s * v for r, s, v in zip(nearby_relation, nearby_strength, nearby_scobility)]
        v_new = sum(nearby_contrib) / sum(nearby_strength)
        # print(f'#{i:3d}: {s}\n>>> {[f"{v:0.6f}" for v in nearby_relation]}\n>>> {[f"{v:8.3f}" for v in nearby_strength]}')
        # print(f'#{i:3d}: {s}\n>>>{scobility_rev[i]:0.6f} -> {v_new:0.6f}\n>>> {[f"{v:0.6f}" for v in nearby_contrib]}')
        scobility_rev[i] = v_new
    scobility_sort = [x for x in zip(scobility_rev, ordering_rev)]
    scobility_sort.sort(key=lambda x: x[0])
    scobility_rev = [x[0] / scobility_sort[0][0] for x in scobility_sort]          # Keep minimum scobility pinned at 0.0
    ordering_rev = [x[1] for x in scobility_sort]
    print(f'>>> Iteration {k+1:2d}: max scobility = {np.log2(scobility_rev[-1]):0.6f}')
    for i, (prev, next) in enumerate(zip(ordering_last, ordering_rev)):
        if prev.s_id != next.s_id:
            pass #print(f'Iteration {k+1:2d}... #{i:3d}: {prev} -> {next}')

In [ ]:
for i, s in enumerate(ordering_rev):
    print(f"{np.log2(scobility_rev[i]):5.3f}🌶️ {str(s):>70s}")

In [ ]:
check_ends = 5
rel_ends = []
for s_lo in ordering_rev[1:check_ends]:
    for s_hi in ordering_rev[-check_ends:-1]:
        r = itl.relationship_lookup(s_lo, s_hi, allow_link=True)
        print(r)
        rel_ends.append(r)

In [ ]:
def quick_plot(r: Relationship):
    x_scores = [r.x.scores[e_id].value for e_id in r.e_common]
    y_scores = [r.y.scores[e_id].value for e_id in r.e_common]

    ex_matrix = 1 - np.vstack([x_scores, y_scores]) * Relationship.SCORE_SCALAR

    screen_max_neg = np.amax(ex_matrix, axis=0)
    screen_min_neg = np.amin(ex_matrix, axis=0)

    ex_matrix_screen = ex_matrix[:, np.logical_and(
        ex_matrix[0, :] < 0.10,
        ex_matrix[1, :] < 0.20,
        screen_min_neg > Relationship.MIN_NEG_LIMIT
    )]

    x_all = ex_matrix_screen[0, :]
    y_all = ex_matrix_screen[1, :]

    plt.scatter(x_all, y_all)
    plt.show()

quick_plot(rel_ends[5])